# Quickstart 2
Demo the use of output parsers.

Lets say that given a customer review, I want the LLM to output some information,but it should be in JSON format.

```json
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [1]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

In [2]:
load_dotenv()

True

In [3]:
def build_prompt(prompt_template, **kwargs):
    prompt_template = ChatPromptTemplate.from_template(prompt_template)
    prompts = prompt_template.format_messages(**kwargs)
    return prompts

In [4]:
def complete(llm, prompts):
    return llm.invoke(prompts)

In [6]:
prompt_template = """For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}"""

customer_review = """This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."""

prompts = build_prompt(prompt_template, text= customer_review)
print(prompts[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


In [7]:
llama2 = ChatOllama(model="llama2", temperature=0)
completion = complete(llama2, prompts)
print(completion.content)
print("\n")
print(type(completion.content))


{
"gift": True,
"delivery_days": 2,
"price_value": ["slightly more expensive than the other leaf blowers out there", "worth it for the extra features"]
}


<class 'str'>


In [8]:
gpt = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  #type: ignore
completion = complete(gpt, prompts)
print(completion.content)
print("\n")
print(type(completion.content))

{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


<class 'str'>


Even though the above output looks like JSON, it is just a plain old string. I cannot use it programmatically. In order to get a real JSON out of the answer, I can use LangChain's output parsers.

In [9]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [10]:
def build_format_instructions(gift, delivery_days, price_value):
    output_parser = StructuredOutputParser.from_response_schemas([
        gift, delivery_days, price_value
    ])
    return output_parser, output_parser.get_format_instructions()

In [11]:
prompt_template = """For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: {text}

{format_instructions}"""


gift = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")

delivery_days = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this information is not found, output -1.")

price_value = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price and output them as a comma separated Python list.")

parser, format_instructions = build_format_instructions(gift, delivery_days, price_value)

prompts = build_prompt(prompt_template, text=customer_review, format_instructions=format_instructions)

print(prompts[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [12]:
llama2 = ChatOllama(model="llama2", temperature=0)
completion = complete(llama2, prompts)
print(completion.content)
print("\n")
print(type(completion.content))
print("\n")
try:
    parser.parse(completion.content)  #type: ignore
except Exception as err:
    print(err)
    print("Notice that LLAMA-2 outputted incorrect JSON, with boolean value as `True` instead of `true`.")

Sure! Here's the extracted information in JSON format:
```json
{
    "gift": True,
    "delivery_days": "2",
    "price_value": ["pretty amazing", "slightly more expensive than the other leaf blowers out there", "worth it for the extra features"]
}
```
Explanation:

* `gift`: Yes, the item was purchased as a gift for someone else.
* `delivery_days`: The product arrived in 2 days.
* `price_value`: The sentences about the value or price of the leaf blower are "pretty amazing", "slightly more expensive than the other leaf blowers out there", and "worth it for the extra features". These are separated by commas because they are a list of values.


<class 'str'>


Got invalid JSON object. Error: Expecting value: line 2 column 13 (char 14)
Notice that LLAMA-2 outputted incorrect JSON, with boolean value as `True` instead of `true`.


In [13]:
gpt = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  #type: ignore
completion = complete(gpt, prompts)
print(completion.content)
print("\n")
print(type(completion.content))
print("\n")
json_content = parser.parse(completion.content)  #type: ignore
print(type(json_content))
print("\n")
print(json_content)

```json
{
	"gift": false,
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


<class 'str'>


<class 'dict'>


{'gift': False, 'delivery_days': '2', 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}
